In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 载入数据
mnist = input_data.read_data_sets("MNIST_data", one_hot=True) # 下载数据

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev) # 标准差
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var) # 直方图

In [4]:
# 批次大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size # // 整除

# 命名空间
with tf.name_scope('input'):
    # 定义两个 placeholder
    x = tf.placeholder(tf.float32, [None, 784], name='x-input') # None 表示任意维度
    y = tf.placeholder(tf.float32, [None, 10], name='y-input')

with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784,10]))
        variable_summaries(W)
    with tf.name_scope('bias'):
        b = tf.Variable(tf.zeros([10]))
        variable_summaries(b)
    with tf.name_scope('wx_bias'):
        wx_plus_bias = tf.matmul(x,W)+b
    with tf.name_scope('softmax'):
        predict = tf.nn.softmax(wx_plus_bias)

# 二次代价函数
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.square(y-predict))
    tf.summary.scalar('loss', loss)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

with tf.name_scope('arruracy'):
    # 结果存放在一个bool 型列表中
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(predict,1)) # tf.equal 
    # 求准确率
    arruracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 合并所有的summary
merged = tf.summary.merge_all()
    
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/', sess.graph)
    for epoch in range(10):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size=batch_size)
            summary, _ = sess.run([merged,train_step], feed_dict={x:batch_xs, y:batch_ys})
        
        writer.add_summary(summary, epoch)
        acc = sess.run(arruracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("Iter" + str(epoch) + "\t Testing Accuracy" + str(acc))
    

Iter0	 Testing Accuracy0.8315
Iter1	 Testing Accuracy0.8698
Iter2	 Testing Accuracy0.8816
Iter3	 Testing Accuracy0.8881
Iter4	 Testing Accuracy0.8942
Iter5	 Testing Accuracy0.8969
Iter6	 Testing Accuracy0.8998
Iter7	 Testing Accuracy0.902
Iter8	 Testing Accuracy0.904
Iter9	 Testing Accuracy0.905
